<a href="https://colab.research.google.com/github/mudogruer/transformers/blob/main/Wav2Vec2_emotion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torchaudio
!pip install -q pandas
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.8 MB/s eta 0:00:00


In [2]:
!pip install -q datasets


In [3]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Speech_dataset")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset

DatasetDict({
    Session1: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1085
    })
    Session2: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1023
    })
    Session3: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1151
    })
    Session4: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1031
    })
    Session5: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1241
    })
})

In [5]:
# Import necessary libraries
from datasets import load_dataset
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC

# Load the dataset
dataset = load_dataset("minoosh/IEMOCAP_Speech_dataset")

# Explore the dataset
print(dataset)

# Initialize the tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base")

# Example of processing a single data point
sample = dataset["Session1"][0]
input_values = tokenizer(sample['audio']['array'], return_tensors="pt").input_values
labels = sample['TURN_NAME']

# Load the model
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base")

# Continue with your training and evaluation logic

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


DatasetDict({
    Session1: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1085
    })
    Session2: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1023
    })
    Session3: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1151
    })
    Session4: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1031
    })
    Session5: Dataset({
        features: ['TURN_NAME', 'audio', 'emotion'],
        num_rows: 1241
    })
})


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:733: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should p

In [6]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

def preprocess_data(batch):
    # Process audio
    audio = batch["audio"]
    input_values = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_values

    # Process labels
    with processor.as_target_processor():
        labels = processor(batch["TURN_NAME"]).input_ids

    batch["input_values"] = input_values
    batch["labels"] = labels
    return batch

dataset = dataset.map(preprocess_data, remove_columns=dataset.column_names["Session1"])


In [7]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Assuming 'dataset' is a Hugging Face dataset
train_test_dataset = dataset["Session3"].train_test_split(test_size=0.2)
train_val_dataset = dataset['Session2'].train_test_split(test_size=0.25)

train_dataset = train_val_dataset
val_dataset = train_val_dataset
test_dataset = train_test_dataset


In [8]:
from transformers import Wav2Vec2Config

# Load configuration for Wav2Vec2
config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base",
                                        output_attentions=False,
                                        output_hidden_states=False)

# Adjust the configuration if necessary


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [15]:
# Access the datasets directly without nesting them under 'train' and 'test'
train_dataset = dataset["Session1"]
val_dataset = dataset["Session2"]
test_dataset = dataset["Session3"]

In [16]:
# Inspect the dataset structure
print("Train Dataset:", train_dataset)
print("Validation Dataset:", val_dataset)
print("Test Dataset:", test_dataset)

# Optionally, inspect a single item to understand its structure
print("Example data point:", train_dataset[0])


Train Dataset: Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1085
})
Validation Dataset: Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1023
})
Test Dataset: Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1151
})
Example data point: {'input_values': [[-0.38823238015174866, -0.38350909948349, -0.29376640915870667, -0.25125670433044434, -0.1993003934621811, -0.12372758984565735, -0.10955768823623657, -0.11664263904094696, -0.09066448360681534, -0.03634653240442276, 0.013248122297227383, 0.015609772875905037, 0.006163171958178282, 0.0486728735268116, 0.03686462342739105, -0.04815478250384331, -0.03870818391442299, 0.013248122297227383, 0.029779674485325813, 0.05575782433152199, 0.022694723680615425, -0.026899930089712143, -0.055239733308553696, -0.12372758984565735, -0.19457709789276123, -0.33155277371406555, -0.4661668539047241, -0.4000406265258789, -0.32210618257522583, -0.39531734585762024, -0.42838042974472046, -0.3339144289493

In [17]:
from transformers import Wav2Vec2Config

# Load configuration for Wav2Vec2
config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base",
                                        output_attentions=False,
                                        output_hidden_states=False)

# Adjust the configuration if necessary


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [23]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
